In [62]:
import pandas as pd
import funzioni as fx
import numpy as np
import yfinance as yf
from tensorflow.keras.models import load_model

def test_trading():
    bilancio = BILANCIO_INIZIALE
    pos_aperta = False
    tp = sl = n_azioni = prezzo_acquisto = prezzo_tot = 0
    giorni_posizione = 0
    posizioni = pd.DataFrame(columns=["Bilancio", "Direzione", "Prezzo_un", "n_azioni", "Prezzo_tot", "TP", "SL", "Open", "High", "Low", "Esito", "P_L"], index=df_X.index)

    for idx, row in df_X.iterrows():
        if bilancio <= 0:
            break
        print(f"\r{idx} Bilancio = {bilancio}                ", end=" ", flush=True)
        if pos_aperta == False:
            if row["prob_True"] > PROBABILITA_PER_ACQUISTO and bilancio > 0:
                #COMPRA
                prezzo_acquisto = row["Open"]
                n_azioni = bilancio // prezzo_acquisto
                prezzo_tot = n_azioni * prezzo_acquisto  
                bilancio -= prezzo_tot          
                tp = prezzo_acquisto * (1 + TP)
                sl = prezzo_acquisto * (1 - SL)
                pos_aperta = True
                giorni_posizione = 0
                posizioni.loc[idx] = {"Bilancio": bilancio, "Direzione": "COMPRA", "Prezzo_un": prezzo_acquisto, "n_azioni": n_azioni, "Prezzo_tot": -prezzo_tot, "TP": tp, "SL": sl, "Open": row["Open"], "High": row["High"], "Low": row["Low"],}
        else:
            if giorni_posizione == 20:
                prezzo_tot_vendita = row['Close'] * n_azioni
                bilancio += prezzo_tot_vendita
                pos_aperta = False
                if prezzo_tot_vendita > prezzo_tot:
                    esito = 'VINCITA'
                else:
                    esito = 'PERDITA'
                pl = fx.pct_change(prezzo_tot, prezzo_tot_vendita).round(0)
                posizioni.loc[idx] = {"Bilancio": bilancio, "Direzione": "VENDI", "Prezzo_un": row['Close'], "n_azioni": n_azioni, "Prezzo_tot": prezzo_tot, "Esito": esito, "Open": row["Open"], "High": row["High"], "Low": row["Low"], "P_L": pl}
            if row["High"] >= tp:
                prezzo_tot_vendita = tp * n_azioni
                bilancio += prezzo_tot_vendita
                pos_aperta = False
                giorni_posizione = 0
                pl = fx.pct_change(prezzo_tot, prezzo_tot_vendita).round(0)
                posizioni.loc[idx] = {"Bilancio": bilancio, "Direzione": "VENDI", "Prezzo_un": tp, "n_azioni": n_azioni, "Prezzo_tot": prezzo_tot, "Esito": "VINCITA", "Open": row["Open"], "High": row["High"], "Low": row["Low"], "P_L": pl}
            elif row["Low"] <= sl:
                prezzo_tot_vendita = sl * n_azioni
                bilancio += prezzo_tot_vendita
                pos_aperta = False
                giorni_posizione = 0
                pl = fx.pct_change(prezzo_tot, prezzo_tot_vendita).round(0)
                posizioni.loc[idx] = {"Bilancio": bilancio, "Direzione": "VENDI", "Prezzo_un": sl, "n_azioni": n_azioni, "Prezzo_tot": prezzo_tot, "Esito": "PERDITA", "Open": row["Open"], "High": row["High"], "Low": row["Low"], "P_L": pl}
            else:
                giorni_posizione += 1

    if pos_aperta: 
        bilancio += prezzo_tot
    posizioni.to_excel("posizioni.xlsx")
    #df_X.to_excel("X.xlsx")
    print(f'\nVincite: {posizioni.loc[posizioni["Esito"] == "VINCITA", "Esito"].count()}, vincita max: {posizioni.loc[posizioni["Esito"] == "VINCITA", "P_L"].max()}')
    print(f'Perdite: {posizioni.loc[posizioni["Esito"] == "PERDITA", "Esito"].count()}, perdita max: {posizioni.loc[posizioni["Esito"] == "PERDITA", "P_L"].min()}')



In [63]:
TP = 0.5
SL = 0.05
PROBABILITA_PER_ACQUISTO = 0.9
BILANCIO_INIZIALE = 1000
SIMBOLO_TEST = 'DHT'

In [64]:
ticker = yf.download(SIMBOLO_TEST, start='2010-01-01', end='2023-12-31', progress=True)
if ticker["Close"].iloc[-1] >= 1:
    ticker.index = ticker.index.date
    print("Calcolo indicatori ticker", flush=True)
    ticker = fx.crea_indicatori(ticker)
    ticker.dropna(axis=0, inplace=True)

    print("Definizione features e target", flush=True)
    _, X, Y, scalers_prezzo = fx.to_XY(ticker, fx.features_prezzo, fx.features_da_scalare_singolarmente, fx.features_meno_piu, fx.features_candele, fx.features_no_scala, fx.elenco_targets, fx.n_timesteps, fx.giorni_previsione, addestramento=False)
print('Previsione')
model = load_model("tuning.keras")
y_pred = model.predict(X)
loss, accuracy, precision, recall, auc = model.evaluate(X, Y)

X = X[:, 59, :len(fx.features_prezzo)]
for i, y in enumerate(y_pred):
    scaler_corrispondente = scalers_prezzo[i]
    y_inv = scaler_corrispondente.inverse_transform(y_pred.reshape(-1, 1))
    X_inv = scaler_corrispondente.inverse_transform(X)
y_pred_true = pd.Series(y_inv.squeeze())

df_X = pd.DataFrame(X_inv, columns=fx.features_prezzo)
df_y_pred = pd.DataFrame(y_pred.squeeze(), columns=['prob_True'])
df_X = pd.concat([df_X, df_y_pred], axis=1)

test_trading()

[*********************100%%**********************]  1 of 1 completed
Calcolo indicatori ticker
Definizione features e target
Previsione
103/103 [==============================] - 34s 297ms/step - loss: 0.3712 - accuracy: 0.8269 - precision: 0.0780 - recall: 0.9600 - auc: 0.9454
3287 Bilancio = 2683.7778377447485                                                                                                                                                                                                   
Vincite: 19, vincita max: 20.0
Perdite: 30, perdita max: -5.0
